### Import the kickstarter scraper

In [4]:
import getpass
from Kickstarter_scraper_to_MySQL import kickstarter_scraper
from sqlalchemy import create_engine

### Set the path to MySQL database

In [5]:
# === Input parameters ===
dbname = 'kickstarter_db2'  # Name of your database
user_id = getpass.getpass('user id:')  # User name
user_password = getpass.getpass('password:')  # Password

# Connection to the MySQL database
engine = create_engine("mysql+pymysql://%s:%s@localhost:3306/%s?charset=utf8"
                       % (user_id, user_password, dbname), encoding='utf8')

table_name_prefix = 'kickstarter' # The name / prefix of table that stores the data
separate_table_for_each_day = False # Whether to build a new table with date as its name, e.g. kickstarter_20170912
restart_drop_table = True # Drop existing table when running the code

# === Run the scraper ===
kickstarter_scraper(engine, table_name_prefix,
                    separate_table_for_each_day=separate_table_for_each_day,
                    restart_drop_table=restart_drop_table)

user id:········
password:········
Database exist? True
Theater
page number = 7, total number of items = 78
Photography
page number = 9, total number of items = 104
Dance
page number = 3, total number of items = 35
Comics
page number = 14, total number of items = 164
Journalism
page number = 6, total number of items = 72
Art
page number = 25, total number of items = 298
Music
page number = 33, total number of items = 395
Games
page number = 47, total number of items = 553
Crafts
page number = 11, total number of items = 132
Design
page number = 44, total number of items = 518
Fashion
page number = 32, total number of items = 381
Food
page number = 23, total number of items = 270
Technology
page number = 47, total number of items = 560
Film_and_Video
page number = 38, total number of items = 445
Publishing
page number = 38, total number of items = 451
number of rows in table "kickstarter" = 4456


,Category,Number of Projects
0,Art,272
1,Comics,150
2,Crafts,116
3,Dance,23
4,Design,501
5,Fashion,356
6,Film_and_Video,421
7,Food,255
8,Games,535
9,Journalism,59


### Check how the database looks like

In [16]:
import pandas as pd
pd.options.display.max_columns = 100

table_name = table_name_prefix
con = engine.connect()

sql_query = """
SELECT * 
FROM %s
LIMIT 5;
"""
kickstarter_data_from_sql = pd.read_sql_query(sql_query % (table_name),con)

kickstarter_data_from_sql

,index,project_launched_at,project_blurb,project_created_at,project_state_changed_at,project_goal,project_slug,project_currency_trailing_code,project_disable_communication,project_is_starrable,project_country,project_pledged,project_static_usd_rate,project_name,project_deadline,project_currency_symbol,project_currency,project_staff_pick,project_backers_count,project_state,creator_slug,creator_id,creator_small,creator_is_registered,creator_name,creator_avatar,location_type,location_state,location_id,location_displayable_name,location_slug,location_urls,location_is_root,location_country,location_short_name,location_localized_name,location_name,profile_background_image_opacity,profile_should_show_feature_image_section,profile_link_url,profile_show_feature_image,profile_text_color,profile_project_id,profile_link_text,profile_link_background_color,profile_background_color,profile_link_text_color,profile_state,profile_state_changed_at,profile_feature_image_attributes,profile_blurb,profile_id,profile_name,category,retrieval_time
0,0,1503058654,The BAFTA award-winning Television Workshop ha...,1484837880,1503058655,15000.0,the-television-workshop-the-new-home,false,false,true,GB,14301.0,1.2905274,The Television Workshop: The New Home,1505650654,£,GBP,false,221,live,Null,1904961866,https://ksr-ugc.imgix.net/assets/017/196/425/f...,true,Nic Harvey,{thumb:https://ksr-ugc.imgix.net/assets/017/19...,Town,England,30720,"Nottingham, UK",nottingham-gb,{web:{discover:https://www.kickstarter.com/dis...,false,GB,"Nottingham, UK",Nottingham,Nottingham,0.8,true,null,false,null,2842967,null,null,null,null,inactive,1484837880,{image_urls:{default:https://ksr-ugc.imgix.net...,null,2842967,null,Theater,12/09/2017 16:00:53
1,0,1503926081,The first ever SHORT marathon that anyone can ...,1501882784,1503926081,8000.0,the-new-york-sorta-marathon,true,false,true,US,3153.99,1.0,The New York Sorta Marathon,1506124800,$,USD,true,69,live,Null,1683069409,https://ksr-ugc.imgix.net/assets/006/039/496/0...,true,Zach Broussard,{thumb:https://ksr-ugc.imgix.net/assets/006/03...,Town,NY,2459115,"New York, NY",new-york-ny,{web:{discover:https://www.kickstarter.com/dis...,false,US,"New York, NY",New York,New York,0.8,true,null,false,null,3097665,null,null,null,null,inactive,1501882784,{image_urls:{default:https://ksr-ugc.imgix.net...,null,3097665,null,Theater,12/09/2017 16:00:53
2,0,1504960630,"A play about family, friendship, control and f...",1500720501,1504960630,5000.0,joy-a-new-play-by-stephanie-martin,false,false,true,GB,1270.0,1.31329077,"\Joy\, a new play by Stephanie Martin",1507552630,£,GBP,false,31,live,Null,138958114,https://ksr-ugc.imgix.net/assets/017/609/339/7...,true,Clamour Theatre Company,{thumb:https://ksr-ugc.imgix.net/assets/017/60...,Town,England,44418,"London, UK",london-gb,{web:{discover:https://www.kickstarter.com/dis...,false,GB,"London, UK",London,London,0.8,true,null,false,null,3082279,null,null,null,null,inactive,1500720501,{image_urls:{default:https://ksr-ugc.imgix.net...,null,3082279,null,Theater,12/09/2017 16:00:53
3,0,1502521600,Thanking You is a Variety Show in Joe's Pub at...,1502000492,1502521601,5000.0,thanking-you-loving-you,true,false,true,US,2285.0,1.0,Thanking You! Loving You!,1505512800,$,USD,false,46,live,thankingyou,1570174670,https://ksr-ugc.imgix.net/assets/017/809/505/a...,true,William Dehorney (Willie Dee),{thumb:https://ksr-ugc.imgix.net/assets/017/80...,County,NY,12589342,"Manhattan, NY",manhattan-ny,{web:{discover:https://www.kickstarter.com/dis...,false,US,"Manhattan, NY",Manhattan,Manhattan,0.8,true,null,false,null,3098868,null,null,null,null,inactive,1502000492,{image_urls:{default:https://ksr-ugc.imgix.net...,null,3098868,null,Theater,12/09/2017 16:00:53
4,0,1505166463,"Due to popular demand, Theatre of Yugen is bri...",1504046867,1505166464,10000.0,the-noh-christmas-carol,true,false,true,US,3526.0,1.0,The Noh Christmas Carol,1510354063,$,USD,false,9,live,theatreofyugen,1300412955,https://ksr-ugc.imgix.net/assets/01

### Sort projects into locations and order them by counts

In [21]:
sql_query = """
SELECT location_state, COUNT(project_name) FROM %s
GROUP BY location_state
ORDER BY COUNT(project_name) DESC
LIMIT 10;
"""
kickstarter_data_from_sql = pd.read_sql_query(sql_query % (table_name),con)

kickstarter_data_from_sql

,location_state,COUNT(project_name)
0,CA,533
1,England,428
2,NY,261
3,TX,178
4,FL,148
5,ON,112
6,IL,107
7,WA,102
8,GA,80
9,OR,72
